In [1]:
import datetime
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools

# Fetch listing

In [8]:
# systems
import os, sys

from dateutil import parser

# scraping websites
import json
import requests
from bs4 import BeautifulSoup

# language processing
from sutime import SUTime

#
sys.path.insert(0, '../src/')
from parse_listing import *

In [9]:
list_id = "mitml"
base_url = "http://mailman.mit.edu/mailman/private/"

local_dir = '../listings/'
local_list_dir = os.path.join(local_dir, list_id)
local_list_dir

# Import URLs and indices
with open(os.path.join(local_list_dir, 'urls.txt'), 'r') as f: 
    urls = [line.rstrip() for line in f.readlines()]
    indices = [os.path.basename(url) for url in urls]

In [13]:
listing = Listing(list_id, "000479.html")

In [15]:
if listing.is_talk: 
    
    datetime_df = listing.get_datetime_predictions()
    listing.get_location()
    
    date = listing.predict_date
    start, end = listing.predict_start, listing.predict_end
    loc = listing.predict_location

In [16]:
print(date, start, end, loc)

2018-08-21 14:00 15:00 E18-304


In [24]:
datetime.strptime('T'.join([date, start]), '%Y-%m-%dT%H:%M').isoformat()

'2018-08-21T14:00:00'

In [33]:
parser.parse('2018-08-21 14:00').isoformat()

'2018-08-21T14:00:00'

# Google Auth instructions
   
Follow [here](https://developers.google.com/calendar/quickstart/python). But save client secrets as `client_secrets.json` and leave `credentials.json` empty. The following code may have to be run via CMD python shell during initialization. 

In [3]:
# Setup the Gmail API
SCOPES =  'https://www.googleapis.com/auth/calendar'

store = file.Storage('../credentials/credentials.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('../credentials/client_secret.json', SCOPES)
    creds = tools.run_flow(flow, store)

In [4]:
service = build('calendar', 'v3', http=creds.authorize(Http()))

Generate new calendar

In [6]:
calendar = {
    'summary': 'mitml',
    'timeZone': 'America/New_York'
}

created_calendar = service.calendars().insert(body=calendar).execute()

print(created_calendar['id'])

7abvvsospg5s9cors0g9bn6veo@group.calendar.google.com


In [27]:
event = {
    'summary': listing.title,
    'location': listing.predict_location,
    'description': listing.url,
    'start': {
        'dateTime': datetime.strptime('T'.join([date, start]), '%Y-%m-%dT%H:%M').isoformat(),
        'timeZone': 'America/New_York',
    },
    'end': {
        'dateTime': datetime.strptime('T'.join([date, end]), '%Y-%m-%dT%H:%M').isoformat(),
        'timeZone': 'America/New_York',
    },
    'attendees': [
        {'email': 'jonathanli6022@gmail.com'}
    ],
    'reminders': {
        'useDefault': False,
        'overrides': [
            {'method': 'email', 'minutes': 24 * 60},
            {'method': 'popup', 'minutes': 10},
        ],
    },
}

event = service.events().insert(calendarId='7abvvsospg5s9cors0g9bn6veo@group.calendar.google.com', body=event).execute()
print('Event created: %s' % (event.get('htmlLink')))

Event created: https://www.google.com/calendar/event?eid=aWVxbzk0OTZiaTVqOHNkamRpa2ZwaXBvbTQgN2FidnZzb3NwZzVzOWNvcnMwZzlibjZ2ZW9AZw
